In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from skimage import io
import numpy as np
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
path_happy= '/content/drive/MyDrive/Cohn-Kanade/happy'
path_surprise = '/content/drive/MyDrive/Cohn-Kanade/surprise'
path_disgust = '/content/drive/MyDrive/Cohn-Kanade/disgust'
path_neutral = '/content/drive/MyDrive/Cohn-Kanade/neutral'
hap_list = os.listdir(path_happy)
surp_list = os.listdir(path_surprise)
neu_list = os.listdir(path_neutral)
disg_list = os.listdir(path_disgust)

X_hap = []
y_hap = []
for i in hap_list :
    X_hap.append(io.imread(os.path.join(path_happy,i),as_gray=True).flatten())
    y_hap.append(0)

X_surp = []
y_surp = []

for i in surp_list :
    X_surp.append(io.imread(os.path.join(path_surprise,i),as_gray=True).flatten())
    y_surp.append(1)

X_disg = []
y_disg = []

for i in disg_list :
    X_disg.append(io.imread(os.path.join(path_disgust,i),as_gray=True).flatten())
    y_disg.append(2)

X_neu = []
y_neu = []

for i in neu_list :
    X_neu.append(io.imread(os.path.join(path_neutral,i),as_gray=True).flatten())
    y_neu.append(3)

X_hap = np.array(X_hap)
X_surp = np.array(X_surp)
X_disg = np.array(X_disg)
X_neu = np.array(X_neu)
X_hap_train, X_hap_test, y_hap_train, y_hap_test = train_test_split(X_hap, y_hap, train_size=60  ,shuffle=True, random_state=21)
X_surp_train, X_surp_test, y_surp_train, y_surp_test = train_test_split(X_surp, y_surp, train_size=60  ,shuffle=True, random_state=21)
X_neu_train, X_neu_test, y_neu_train, y_neu_test = train_test_split(X_neu, y_neu, train_size=120  ,shuffle=True, random_state=21)
X_disg_train, X_disg_test, y_disg_train, y_disg_test = train_test_split(X_disg, y_disg, train_size=30  ,shuffle=True, random_state=21)
X_train = np.concatenate((X_hap_train,X_surp_train,X_neu_train,X_disg_train))
X_test = np.concatenate((X_hap_test,X_surp_test,X_disg_test,X_neu_test))
y_train = np.concatenate((y_hap_train,y_surp_train,y_neu_train,y_disg_train))
y_test = np.concatenate((y_hap_test,y_surp_test,y_disg_test,y_neu_test))

Dataset is ready


In [ ]:
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=100)
pca.fit(X_train-np.mean(X_train,axis=0))

PCA(n_components=100)

In [ ]:
x_train_PCA =  pca.transform(X_train-np.mean(X_train,axis=0))
x_test_PCA = pca.transform(X_test-np.mean(X_train,axis=0))

Complexity Reduction using PCA

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
X, y = make_classification(random_state=0)
clf = make_pipeline(StandardScaler(),LinearSVC(C=100,random_state=0, tol=1e-5))
clf.fit(x_train_PCA,y_train)
y_pred = clf.predict(x_test_PCA)

matrix = confusion_matrix(y_test,y_pred, labels=[3,2,1,0])
print('Confusion matrix : \n',matrix)

tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[3,2,1,0])
print('Outcome values : \n', tp, fn, fp, tn)

matrix = classification_report(y_test,y_pred,labels=[3,2,1,0])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[25  8  9 28]
 [ 7  3  1  6]
 [12  1  9  6]
 [13  1  2  9]]
Outcome values : 
 [25  8  9 28] [7 3 1 6] [12  1  9  6] [13  1  2  9]
Classification report : 
               precision    recall  f1-score   support

           3       0.44      0.36      0.39        70
           2       0.23      0.18      0.20        17
           1       0.43      0.32      0.37        28
           0       0.18      0.36      0.24        25

    accuracy                           0.33       140
   macro avg       0.32      0.30      0.30       140
weighted avg       0.37      0.33      0.34       140



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
from sklearn.utils import shuffle
x_train,y_train = shuffle(x_train_PCA,y_train)
x_train_split = np.array(np.split(x_train,5))
y_train_split = np.array(np.split(y_train,5 ))
print(x_train_split.shape)

(5, 54, 100)


In [ ]:
C = [0.01, 0.1, 1, 10, 100]
mean = []
for c in C : 
    s = []
    for i in [0,1,2,3,4]:
        f = [0,1,2,3,4]
        f.remove(i)
        x_test_fold1 = x_train_split[i , : , :]
        y_test_fold1 = y_train_split[i , :]
        x_train_fold1 = x_train_split[f, :]
        x_train_fold1 = x_train_fold1.reshape(x_train_fold1.shape[0]*x_train_fold1.shape[1] , x_train_fold1.shape[2])
        y_train_fold1 = y_train_split[f , :].flatten()
        svc = LinearSVC(C = c , multi_class='ovr')
        svc.fit(x_train_fold1,y_train_fold1)
        s.append(svc.score(x_test_fold1,y_test_fold1))
    mean.append(np.mean(s))
C = C[np.argmax(mean)]

In [ ]:
print(f'optimal C is :', C )

optimal C is : 0.01


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
X, y = make_classification(random_state=0)
clf = make_pipeline(StandardScaler(),LinearSVC(C=0.01,random_state=0, tol=1e-5))
clf.fit(x_train_PCA,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=0.01, random_state=0, tol=1e-05))])

In [ ]:
y_pred = clf.predict(x_test_PCA)
y_test.shape

(140,)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


matrix = confusion_matrix(y_test,y_pred, labels=[3,2,1,0])
print('Confusion matrix : \n',matrix)

tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[3,2,1,0])
print('Outcome values : \n', tp, fn, fp, tn)

matrix = classification_report(y_test,y_pred,labels=[3,2,1,0])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[41  5 13 11]
 [ 5  2  4  6]
 [18  1  5  4]
 [17  3  4  1]]
Outcome values : 
 [41  5 13 11] [5 2 4 6] [18  1  5  4] [17  3  4  1]
Classification report : 
               precision    recall  f1-score   support

           3       0.51      0.59      0.54        70
           2       0.18      0.12      0.14        17
           1       0.19      0.18      0.19        28
           0       0.05      0.04      0.04        25

    accuracy                           0.35       140
   macro avg       0.23      0.23      0.23       140
weighted avg       0.32      0.35      0.33       140



### as we see , accuracy rises , and new model is better than previous which we trained without insight on C .

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'gamma':[0.000001,0.000004,0.000006,0.000008,0.00001], 'C':[0.01 , 0.1 , 1 , 10 , 100]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(x_train_PCA, y_train)
clf.best_params_

{'C': 0.01, 'gamma': 1e-06}

### Grid search also approves our implemented algorithm for finding C and recommends us the same C as we obtained in the last part .


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(C=0.01,kernel = 'rbf' ,gamma=1e-8))
clf.fit(x_train_PCA,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.01, gamma=1e-08))])

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred = clf.predict(x_test_PCA)

matrix = confusion_matrix(y_test,y_pred, labels=[3,2,1,0])
print('Confusion matrix : \n',matrix)

tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[3,2,1,0])
print('Outcome values : \n', tp, fn, fp, tn)

matrix = classification_report(y_test,y_pred,labels=[3,2,1,0])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[70  0  0  0]
 [17  0  0  0]
 [28  0  0  0]
 [25  0  0  0]]
Outcome values : 
 [70  0  0  0] [17  0  0  0] [28  0  0  0] [25  0  0  0]
Classification report : 
               precision    recall  f1-score   support

           3       0.50      1.00      0.67        70
           2       0.00      0.00      0.00        17
           1       0.00      0.00      0.00        28
           0       0.00      0.00      0.00        25

    accuracy                           0.50       140
   macro avg       0.12      0.25      0.17       140
weighted avg       0.25      0.50      0.33       140



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### since RBF kernel can classify non-separable Datasets , We expect to have better model with RBF kernel . As we see Accuracy has been better with RBF kernel rather than linear SVM . 